In [44]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import random
import itertools
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim

np.random.seed(0)

In [45]:
# definição dos grupos de filiais pra facilitar a vida

df = pd.read_csv("dados_simulacao/distancias_todas_combinacoes.csv")

CD = 'CENTRO DE DISTRIBUICAO'

todas_com_cd = df

todas_sem_cd = df[(df['Filial_A'] != CD) 
                      & (df['Filial_B'] != CD)]

rj_com_cd = df[(df['Estado_A'] == 'RJ')|(df['Estado_B'] == 'RJ')]  # uniao de E_A e E_B == RJ

rj_sem_cd = todas_sem_cd[(todas_sem_cd['Estado_A'] == 'RJ')|(todas_sem_cd['Estado_B'] == 'RJ')]  # uniao de E_A e E_B == RJ \{filial = CD}

sp_sem_cd = todas_sem_cd[(todas_sem_cd['Estado_A'] == 'SP')|(todas_sem_cd['Estado_B'] == 'SP')] # uniao de E_A e E_B == SP

cd_sp = df[((df['Estado_A'] == 'SP') & (df['Filial_B'] == CD))
           | ((df['Estado_B'] == 'SP') & (df['Filial_A'] == CD))]
sp_com_cd = pd.concat((cd_sp, sp_sem_cd), ignore_index=True)  # uniao de E_A e E_B == SP  +  {A = CD e E_B = SP}

lojas_A = todas_sem_cd[['Estado_A', 'Filial_A']].rename(columns={'Estado_A': 'Estado', 'Filial_A': 'Filial'})
lojas_B = todas_sem_cd[['Estado_B', 'Filial_B']].rename(columns={'Estado_B': 'Estado', 'Filial_B': 'Filial'})

um_de_cada = pd.concat([lojas_A, lojas_B]) \
                     .drop_duplicates() \
                     .sort_values(by=['Estado', 'Filial']) \
                     .drop_duplicates(subset=['Estado']) \
                     .reset_index(drop=True)

estado_sem_cd = todas_sem_cd[(todas_sem_cd['Filial_A'].isin(um_de_cada['Filial'].tolist())) &
                             (todas_sem_cd['Filial_B'].isin(um_de_cada['Filial'].tolist()))]

cd_filiais = df[(df['Filial_A'] == CD) & (df['Filial_B'].isin(um_de_cada['Filial'].tolist()))]

estado_com_cd = pd.concat([estado_sem_cd, cd_filiais], ignore_index=True)

___

#### Temos uma função $F_A(B)$ = valor de B no ranking de A:

##### 1) Caso A→B (A como provedor):

$$F_A(B) = (\frac{1}{Leadtime} + \frac{1}{CustoFrete}) * (ContRup*p + ContFalta*q)$$

##### 2) Caso CD→B (CD como provedor): 

$$F_{CD}(B) = α * (\frac{1}{Leadtime} + \frac{1}{CustoFrete}) * (ContRup*p + ContFalta*q)$$

onde $α$ é um fator preferencia do CD ser o provedor.

___

#### Utilize uma das variáveis para a simulação:

* `todas_sem_cd` para utilizar todas as filiais do Brasil.
* `todas_com_cd` para utilizar todas as filiais do Brasil e o Centro de Distribuição.
* `rj_sem_cd` para utilizar todas as filiais do Rio de Janeiro.
* `rj_com_cd` para utilizar todas as filiais do Rio de Janeiro e o Centro de Distribuição.
* `sp_sem_cd` para utilizar todas as filiais de São Paulo.
* `sp_com_cd` para utilizar todas as filiais de São Paulo e o Centro de Distribuição.
* `estado_sem_cd` para utilizar uma filial e cada estado.
* `estado_com_cd` para utilizar uma filial e cada estado e o Centro de Distribuição.

#### Não esqueça de especificar se o CD está incluso ou não (True/False)

In [46]:
# Defina todas as variáveis abaixo!

filiais = estado_sem_cd # qual conjunto de variáveis usar 
cd = False # booleano se considera ou não o CD no grafo # acho que nem vai ser necessário

c = 1 # constante de tempo de transporte por km 
k_filial = 0.5  # constante de custo do frete em reais por km saindo de A
k_cd = 0.2  # constante de custo do frete em reais por km saindo do CD
p = 2  # peso se a filial está em ruptura
q = 1  # peso se a filial está em falta
alpha = 2  # fator de preferencia do CD ser o provedor

params = {'c': c, 'k_filial': k_filial, 'k_cd': k_cd, 'p': p, 'q': q, 'alpha': alpha}


In [ ]:
def classificar_filiais(df):
    """
    Classifica um DataFrame em DataFrames de provedores e receptoras.

    Argumentos:
    df -- DataFrame com as colunas: 
                 ['Filial', 'Tipo', 'Est_disp', 'Alvo']
    
    Retorna:
    (df_provedores, df_receptoras) -- Uma tupla contendo os dois DataFrames.
    """
   
    # Filais provedoras
    df_prov = df.copy()
    df_prov['Vol_Exc'] = np.where((df_prov['Tipo'] == 'CD'), # se é o CD
                                   df_prov['Est_disp'],      # o valor do excesso = estoque disponivel
                                   (df_prov['Est_disp'] - df_prov['Alvo']).clip(lower=0))   # se não, a diferença entre estoque e alvo 
    df_provedores = df_prov[df_prov['Vol_Exc'] > 0].copy()
    df_provedores = df_provedores[['Filial', 'Tipo', 'Vol_Exc']]

    # Filiais receptoras
    df_rec = df[df['Tipo'] == 'Filial'].copy()
    df_receptoras = df_rec[df_rec['Est_disp'] < df_rec['Alvo']].copy() # pega só as com estoque abaixo do alvo
    df_receptoras['Vol_Falta'] = (df_receptoras['Alvo'] - df_receptoras['Est_disp']) # Vol_Falta é o quanto falta para chegar no alvo
    df_receptoras['Cont_Rup'] = np.where(df_receptoras['Est_disp'] == 0, 1, 0) # 
    df_receptoras['Cont_Falta'] = np.where(df_receptoras['Est_disp'] > 0, 1, 0)

    colunas_finais_rec = ['Filial', 'Vol_Falta', 'Cont_Rup', 'Cont_Falta']
    df_receptoras = df_receptoras[colunas_finais_rec]
    
    return df_provedores, df_receptoras

In [48]:
def get_distancia(a, b, filiais):
    """Função para buscar distâncias no df"""
    if a == b:
        return 0
    dist_df = filiais[
        ((filiais['Filial_A'] == a) & (filiais['Filial_B'] == b)) |
        ((filiais['Filial_A'] == b) & (filiais['Filial_B'] == a))
    ]
    if dist_df.empty:
        return np.inf  # Retorna infinito se a rota não existe
    return dist_df['Distancia_km'].values[0]

In [ ]:
def calcular_ranking_F(
    provedor, 
    receptor, 
    status_receptor, 
    filiais, 
    params
):
    """Calcula o score F_A(B) ou F_CD(B) para um par (A, B)."""
    
    dist = get_distancia(provedor, receptor, filiais)
    if dist == np.inf or dist == 0:
        return 0  # Score 0 se a rota não existe ou é para si mesmo

    leadtime = params['c'] * dist  # calculo do leadtime
    
    if provedor == CD:
        custo_frete = params['k_cd'] * dist # calculo do custo (CD)
    else:
        custo_frete = params['k_filial'] * dist # calculo do custo (filial)

    if status_receptor == "Ruptura":
        cont_rup = 1
        cont_falta = 0
    elif status_receptor == "Falta":
        cont_rup = 0
        cont_falta = 1
    else:
        return 0 
    # finalmente o calculo da formula 
    f_AB = ((1 / (leadtime+1)) + (1 / (custo_frete+1))) * ((cont_rup * params['p']) + (cont_falta * params['q']))
    
    if provedor == CD: # multiplica por alpha se o CD for o provedor 
        return params['alpha'] * f_AB
    else:
        return f_AB

In [ ]:
def montar_matriz_T(lista_provedores, df_receptoras, filiais, params):
    """
    Monta a Matriz T (DataFrame) onde T[i, j] = F_Ai(B_j).
    
    - lista_provedores: Uma lista de nomes de filiais provedoras (ex: ['CD', 'A1', 'A2'])
    - df_receptoras: O DataFrame de filiais B (receptoras)
    """
    
    matriz_data = []
    
    # Itera sobre cada receptoras (colunas)
    for _, receptor in df_receptoras.iterrows():
        filial_b = receptor['Filial']
        status_b = receptor['Status']
        
        scores_para_b = {'Receptora': filial_b}
        
        # Itera sobre cada PROVEDOR (linhas)
        for filial_a in lista_provedores:
            
            # Calcula o score do par (A, B)
            score = calcular_ranking_F(
                provedor=filial_a,
                receptor=filial_b,
                status_receptor=status_b,
                filiais=filiais,
                params=params
            )
            scores_para_b[filial_a] = score
            
        matriz_data.append(scores_para_b)
        
    # Converte os dados em um DataFrame e o "pivota" (transpõe)
    # Queremos Provedores (A_i) nas linhas e Receptoras (B_j) nas colunas
    
    matriz_T = pd.DataFrame(matriz_data)
    matriz_T = matriz_T.set_index('Receptora').T
    
    return matriz_T

In [51]:
# --- EXEMPLO DE USO ---

# 1. Definir seus parâmetros
params = {
    'c': 0.5, 'k_filial': 2.0, 'k_cd': 1.0, 'p': 10, 'q': 5, 'alpha': 1.5
}
CD_NOME = "CD-SP"

# 2. Carregar seus dados (simulados aqui)
lista_provedores = ["CD-SP", "Filial-RJ", "Filial-MG"]
df_receptoras_simulado = pd.DataFrame([
    {'Filial': 'Filial-BA', 'Produto': 'SKU-1', 'Qtd_Necessaria': 10, 'Status': 'Ruptura'},
    {'Filial': 'Filial-RS', 'Produto': 'SKU-1', 'Qtd_Necessaria': 5, 'Status': 'Falta'},
    {'Filial': 'Filial-AM', 'Produto': 'SKU-1', 'Qtd_Necessaria': 20, 'Status': 'Ruptura'}
])
df_distancias_simulado = pd.DataFrame([
    {'Filial_A': 'CD-SP', 'Filial_B': 'Filial-RJ', 'Distancia_km': 450},
    {'Filial_A': 'CD-SP', 'Filial_B': 'Filial-MG', 'Distancia_km': 580},
    {'Filial_A': 'CD-SP', 'Filial_B': 'Filial-BA', 'Distancia_km': 1900},
    {'Filial_A': 'CD-SP', 'Filial_B': 'Filial-RS', 'Distancia_km': 1100},
    {'Filial_A': 'CD-SP', 'Filial_B': 'Filial-AM', 'Distancia_km': 4000},
    {'Filial_A': 'Filial-RJ', 'Filial_B': 'Filial-BA', 'Distancia_km': 1600},
    {'Filial_A': 'Filial-RJ', 'Filial_B': 'Filial-RS', 'Distancia_km': 1550},
    # ... etc ...
])


# 3. Montar a Matriz T
matriz_T = montar_matriz_T(
    lista_provedores, 
    df_receptoras_simulado, 
    df_distancias_simulado, 
    params, 
    CD_NOME
)

print("--- Matriz T (Scores F_A(B)) ---")
print(matriz_T)

# 4. Encontrar o melhor provedor para cada B (seu passo 3)
# "pegamos o maior valor de cada coluna"
# .idxmax() encontra o índice (nome do provedor) do maior valor na coluna
melhores_provedores_para_B = matriz_T.idxmax(axis=0)

print("\n--- Melhor Provedor para cada Receptora (B escolhe A) ---")
print(melhores_provedores_para_B)

# 5. Criar as listas de distribuição para cada A (seu passo 5)
# (Isso cria o grafo bipartido que você mencionou: {A_1:[B_3], A_2:[B_2,B_1], ...})

atribuicoes = {}
for b, a in melhores_provedores_para_B.items():
    if a not in atribuicoes:
        atribuicoes[a] = []
    
    # Pega o score que B deu para A
    score = matriz_T.loc[a, b] 
    atribuicoes[a].append({'receptora': b, 'score': score})

# Ordenar as listas de B para cada A (maior score primeiro)
for a in atribuicoes:
    atribuicoes[a].sort(key=lambda x: x['score'], reverse=True)

print("\n--- Listas de Alocação (A distribui para B) ---")
print(atribuicoes)

# 6. O passo final seria a função de alocação (seu passo 6 e 7)
# Isso exigiria o df_provedores (com Estoque_Excedente)
# e o df_receptoras (com Qtd_Necessaria)
# e faria um loop sobre 'atribuicoes' para distribuir o estoque.

--- Matriz T (Scores F_A(B)) ---
Receptora  Filial-BA  Filial-RS  Filial-AM
CD-SP       0.013158   0.011364    0.00625
Filial-RJ   0.015625   0.008065    0.00000
Filial-MG   0.000000   0.000000    0.00000

--- Melhor Provedor para cada Receptora (B escolhe A) ---
Receptora
Filial-BA    Filial-RJ
Filial-RS        CD-SP
Filial-AM        CD-SP
dtype: object

--- Listas de Alocação (A distribui para B) ---
{'Filial-RJ': [{'receptora': 'Filial-BA', 'score': np.float64(0.015625)}], 'CD-SP': [{'receptora': 'Filial-RS', 'score': np.float64(0.011363636363636364)}, {'receptora': 'Filial-AM', 'score': np.float64(0.00625)}]}
